In [2]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Airline Performance").getOrCreate()

In [ ]:
#read records and make sql dataframe
flights = spark.read.option("header",True).csv('hdfs:///airline_performance/airline_2m.csv')
flights.createOrReplaceTempView("flights")

In [5]:
#find airports as nodes in graph
airports = spark.sql("select Origin as airport from flights union select Dest from flights")

In [6]:
#(origin airport, dest airport, airline) key stats as edges
stats = spark.sql("select Origin as src, Dest as dst, Reporting_Airline as relationship, sum(cast(DepDelay as int)) as Departure_Delay, sum(cast(ArrDelay as int)) as Arrival_Delay, sum(cast(ActualElapsedTime as int) - cast(CRSElapsedTime as int)) as Wasted_Time, sum(cast(Cancelled as int)) as Cancelled, sum(cast(Diverted as int)) as Diverted, count(*) as Flights_Count from flights group by Origin, Dest, Reporting_Airline")

In [ ]:
airports.coalesce(1).write.csv("hdfs:///airline_performance/dest")